In [2]:
import pyltr
import pandas as pd
import numpy as np
import MRR
import csv
import time
import dawg
import pickle


In [3]:
training_file = "../data/training_feature_data.csv"

In [4]:
num_training_lines = 106161931
num_validation_lines = 102964961
amount_of_divisions = 20

preallocate_training = int(num_training_lines / (amount_of_divisions))
preallocate_validation = int(num_validation_lines / (amount_of_divisions))

In [5]:
# Maak klaar voor andere feature krijgen

with open('../data/sorted_popular_queries.csv', mode='r') as infile:
    reader = csv.reader(infile)
    skipheader = next(reader)
    data=[tuple([line[1], int(line[2])]) for line in reader if int(line[2])>2]
sortedpopulardict = dawg.IntDAWG(data)

In [ ]:
# training_data = pd.read_csv("../data/training_feature_data.csv", header=None)

start_time = time.time()
training_ngram_features = []
training_ylabels = []
training_qids = []

training_ngram_features=preallocate_training*[[0,0,0,0,0,0]]
training_ylabels=preallocate_training*[np.bool(0)]
training_qids=preallocate_training*[np.int(0)]
training_popular_count_feature = preallocate_training*[np.int(0)]
training_word_length_feature = preallocate_training*[np.int(0)]
training_char_length_feature = preallocate_training*[np.int(0)]
training_lastspace_feature=preallocate_training*[np.bool(0)]

i = 0
with open(training_file, newline='\n') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    prev_qid = -1
    current_qid = -1
    split_counter = 0
    
    for row in spamreader:
        if int(row[3]) % amount_of_divisions == 0:
            continue
        
        # Get features
        training_ngram_features[i] = eval(row[0])
        training_popular_count_feature[i] = sortedpopulardict.get(row[2], 0)
        training_word_length_feature[i] = len(row[2].split())
        training_char_length_feature[i] = len(row[2])
        training_lastspace_feature[i] = row[2][training_char_length_feature[i] - 1] == ' '

        # Set question and label
        training_ylabels[i] = np.bool(int(row[1]))
        training_qids[i] = int(row[3])
        i = i + 1
        if i == preallocate_training:
            break
            
        if i % 1000000 == 0:
            print(str(i) + "/ " + str(preallocate_training))
        
            
training_ngram_ints = np.array(training_ngram_features)
training_ylabels_ready = np.array(training_ylabels)
training_qids_ready = np.array(training_qids)

training_ngram_features = []
training_ylabels = []
training_qids = []

print("Dit koste: " + str(time.time() - start_time))

1000000/ 5308096
2000000/ 5308096
3000000/ 5308096


In [ ]:
validation_file = "../data/validation_feature_data.csv"

In [ ]:
# validation_data = pd.read_csv("../data/validation_feature_data.csv", header=None)

start_time = time.time()
validation_ngram_features = []
validation_ylabels = []
validation_qids = []

validation_ngram_features=preallocate_validation*[[0,0,0,0,0,0]]
validation_ylabels=preallocate_validation*[np.bool(0)]
validation_qids=preallocate_validation*[np.int(0)]
validation_popular_count_feature = preallocate_validation*[np.int(0)]
validation_word_length_feature = preallocate_validation*[np.int(0)]
validation_char_length_feature = preallocate_validation*[np.int(0)]
validation_lastspace_feature=preallocate_validation*[np.bool(0)]

i = 0
with open(validation_file, newline='\n') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    prev_qid = -1
    current_qid = -1
    split_counter = 0
    
    for row in spamreader:
        if int(row[3]) % amount_of_divisions == 0:
            continue
        
        # Get features
        validation_ngram_features[i] = eval(row[0])
        validation_popular_count_feature[i] = sortedpopulardict.get(row[2], 0)
        validation_word_length_feature[i] = len(row[2].split())
        validation_char_length_feature[i] = len(row[2])
        validation_lastspace_feature[i] = row[2][validation_char_length_feature[i] - 1] == ' '

        # Set question and label
        validation_ylabels[i] = np.bool(int(row[1]))
        validation_qids[i] = int(row[3])
        i = i + 1
        if i == preallocate_validation:
            break
        
        if i % 1000000 == 0:
            print(str(i) + "/ " + str(preallocate_validation))
            
validation_ngram_ints = np.array(validation_ngram_features)
validation_ylabels_ready = np.array(validation_ylabels)
validation_qids_ready = np.array(validation_qids)

validation_ngram_features = []
validation_ylabels = []
validation_qids = []

print("Dit koste: " + str(time.time() - start_time))

In [ ]:
# Merging all the features, ngram, word length, char length, popularity & last space

training_features_final = np.ndarray(shape=(len(training_ngram_ints), 10))
for i in range(len(training_ngram_ints)):
    training_features_final[i] = np.append(training_ngram_ints[i], [training_popular_count_feature[i], training_word_length_feature[i], training_char_length_feature[i], training_lastspace_feature[i]])

del training_ngram_ints
del training_popular_count_feature
del training_word_length_feature
del training_char_length_feature
del training_lastspace_feature
import gc
gc.collect()

In [ ]:
# Merging all the features, ngram, word length, char length, popularity & last space

validation_features_final = np.ndarray(shape=(len(validation_ngram_ints), 10))
for i in range(len(validation_ngram_ints)):
    validation_features_final[i] = np.append(validation_ngram_ints[i], [validation_popular_count_feature[i], validation_word_length_feature[i], validation_char_length_feature[i], validation_lastspace_feature[i]])

del validation_ngram_ints
del validation_popular_count_feature
del validation_word_length_feature
del validation_char_length_feature
del validation_lastspace_feature
import gc
gc.collect()

In [ ]:
metric = pyltr.metrics.ERR(k=10, highest_score = 1)
metric_MRR = MRR.MRR(k=10, highest_score = 1)

# Only needed if you want to perform validation (early stopping & trimming)
monitor = pyltr.models.monitors.ValidationMonitor(
    validation_features_final, validation_ylabels_ready, validation_qids_ready, metric=metric, stop_after=5)

model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=300,
    verbose=1
)

model.fit(training_features_final, training_ylabels_ready, training_qids_ready, monitor=monitor)

In [ ]:
model.estimators_

In [ ]:
pickle.dump( model, open( "model1top100k5iter.p", "wb" ) )

In [ ]:
# # testing_data = pd.read_csv("../data/testing_feature_data.csv", header=None)
testing_file = "../data/testing_feature_data.csv"
start_time = time.time()
testing_ngram_features = []
testing_ylabels = []
testing_qids = []
preallocate_testing = 26821328
testing_ngram_features=preallocate_testing*[[0,0,0,0,0,0]]
testing_ylabels=preallocate_testing*[np.bool(0)]
testing_qids=preallocate_testing*[np.int(0)]
testing_popular_count_feature = preallocate_testing*[np.int(0)]
testing_word_length_feature = preallocate_testing*[np.int(0)]
testing_char_length_feature = preallocate_testing*[np.int(0)]
testing_lastspace_feature=preallocate_testing*[np.bool(0)]

i = 0
with open(testing_file, newline='\n') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    prev_qid = -1
    current_qid = -1
    split_counter = 0
    
    for row in spamreader:
        # Get features
        testing_ngram_features[i] = eval(row[0])
        testing_popular_count_feature[i] = sortedpopulardict.get(row[2], 0)
        testing_word_length_feature[i] = len(row[2].split())
        testing_char_length_feature[i] = len(row[2])
        testing_lastspace_feature[i] = row[2][testing_char_length_feature[i] - 1] == ' '

        # Set question and label
        testing_ylabels[i] = np.bool(int(row[1]))
        testing_qids[i] = int(row[3])
        i = i + 1
        
            
testing_ngram_ints = np.array(testing_ngram_features)
testing_ylabels_ready = np.array(testing_ylabels)
testing_qids_ready = np.array(testing_qids)

testing_ngram_features = []
testing_ylabels = []
testing_qids = []

print("Dit koste: " + str(time.time() - start_time))



In [ ]:
# Merging all the features, ngram, word length, char length, popularity & last space

testing_features_final = np.ndarray(shape=(len(testing_ngram_ints), 10))
for i in range(len(testing_ngram_ints)):
    testing_features_final[i] = np.append(testing_ngram_ints[i], [testing_popular_count_feature[i], testing_word_length_feature[i], testing_char_length_feature[i], testing_lastspace_feature[i]])

del testing_ngram_ints
del testing_popular_count_feature
del testing_word_length_feature
del testing_char_length_feature
del testing_lastspace_feature
import gc
gc.collect()

In [ ]:
Epred = model.predict(testing_features_final)
print('Our model:' +  str(metric.calc_mean(testing_qids_ready, testing_ylabels_ready, Epred)))

In [ ]:
print(' Our model MRR:'+  str(metric_MRR.calc_mean(testing_qids_ready, testing_ylabels_ready, Epred)))

In [ ]:
np.bool(int('0'))